In [35]:
from datetime import datetime
import seaborn as sns
import pandas as pd
import numpy as np
import folium 
import matplotlib.pyplot as plt
import platform
from datetime import datetime
import warnings
import os
import glob
warnings.filterwarnings("ignore")
plt.rcParams["font.family"] = 'AppleGothic'  # 사용할 한글 폰트 설정

### 데이터 불러오기

In [36]:
# uriage.csv: 매출이력(기간: 2019/1월 ~ 2019/7월)
# kokyaku_daicho.xlsx : 대리점에서 관리하는 고객정보
# dump 고객 구매정보
dump = pd.read_csv('./Data-in/dump_data.csv')
kokyaku = pd.read_excel('./Data-in/kokyaku_daicho.xlsx')
uriage = pd.read_csv('./Data-in/uriage.csv')
print(dump.head())
print(kokyaku.head())
print(uriage.head())

         purchase_date  purchase_month item_name  item_price 고객이름  지역  \
0  2019-06-13 18:02:00          201906       상품A       100.0  김가온  C시   
1  2019-07-13 13:05:00          201907       상품S      1900.0  김우찬  C시   
2  2019-05-11 19:42:00          201905       상품A       100.0  김유찬  A시   
3  2019-02-12 23:40:00          201902       상품Z      2600.0  김재현  D시   
4  2019-04-22 03:09:00          201904       상품A       100.0  김강현  D시   

                   등록일  
0  2017-01-26 00:00:00  
1  2018-04-07 00:00:00  
2  2018-06-19 00:00:00  
3  2018-07-22 00:00:00  
4  2017-06-07 00:00:00  
   고객이름  지역                  등록일
0  김 현성  H시  2018-01-04 00:00:00
1  김 도윤  E시                42782
2  김 지한  A시  2018-01-07 00:00:00
3  김 하윤  F시                42872
4  김 시온  E시                43127
      purchase_date item_name  item_price customer_name
0  2019-06-13 18:02       상품A       100.0           김가온
1  2019-07-13 13:05     상 품 S         NaN           김우찬
2  2019-05-11 19:42     상 품 a         NaN    

### 데이터 오류 살펴보기

In [37]:
print(dump.isna().sum())
print('#################')
print(kokyaku.isna().sum())
print('#################')
print(uriage.isna().sum()) # item_price 387개의 결측치 발견

purchase_date     0
purchase_month    0
item_name         0
item_price        0
고객이름              0
지역                0
등록일               0
dtype: int64
#################
고객이름    0
지역      0
등록일     0
dtype: int64
#################
purchase_date      0
item_name          0
item_price       387
customer_name      0
dtype: int64


In [38]:
uriage.loc[uriage['item_price'].isna()]

,purchase_date,item_name,item_price,customer_name
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
4,2019-04-22 3:09,상품a,NaN,김강현
6,2019-05-18 19:16,상품 a,NaN,김재준
14,2019-06-11 12:57,상품A,NaN,김재호
...,...,...,...,...
2987,2019-01-04 13:05,상품K,NaN,김정훈
2990,2019-07-16 11:34,상품O,NaN,김서준
2992,2019-07-27 10:13,상품C,NaN,김동하
2996,2019-03-29 11:14,상품Q,NaN,김지율


### 데이터에 오류가 있는 상태로 집계

In [39]:
# '매출 이력'에서 상품별로 월 매출 합계를 집계
# index : month_year(년월) , columns : item_name, size
uriage['purchase_date'] = pd.to_datetime(uriage['purchase_date'])
uriage['month_year'] = uriage['purchase_date'].dt.strftime('%Y%m')
pd.pivot_table(
    data = uriage, 
    index = 'month_year',
    columns = 'item_name',
    values = 'item_price',
    aggfunc='size',
    fill_value=0
)

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
month_year,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [40]:
pd.pivot_table(
    data = uriage, 
    index = 'month_year',
    columns = 'item_name',
    values = 'item_price',
    aggfunc='sum',
    fill_value=0
)

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
month_year,,,,,,,,,,,,,,,,,,,,,
201901,1400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1100.0,1200.0,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201902,0.0,0.0,0.0,0.0,0.0,0.0,2400.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,2000.0,2200.0,0.0,0.0
201903,0.0,500.0,1300.0,1600.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2300.0,0.0,1700.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0
201905,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2500.0
201906,0.0,0.0,0.0,0.0,0.0,2300.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1600.0,0.0,0.0,0.0,0.0,2400.0,0.0
201907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1500.0,0.0,1800.0,0.0,0.0,0.0,0.0,0.0


### 상품명 오류 수정

In [41]:
uriage['item_name'].unique() #상품명 형식이 일정하지가 않음

array(['상품A', '상 품 S', '상 품 a', '상품Z', '상품a', '상품S', '상품 a', '상품V', '상품O',
       '상 품U', '상품L', '상  품V', '상 품O', '상품C', '상품I', '상품r', '상품X', '상품 g',
       '상품R', '상품P', '상품Q', '상품y', '상품 A', '상품N', '상품W', '상 품E', '상품K',
       '상품B', '상품F', '상 품s', ' 상품W', ' 상 품 n', '상 품F', '상품D', '상품M',
       '상품Y', '상품U', '상품H', '상품T', '상품J', '상  품O', '상품E', '상  품Q',
       ' 상품 S', ' 상품 M', '상  품T', '상품G', '상 품G', ' 상품 P', ' 상품 E', '상 품N',
       '상 품Y', '상품 J', '상품 V', '상품 K', '상 품V', '상 품D', '상 품A', '상품 F',
       '상품 H', '상 품K', '상 품T', '상품 X', '상품 Q', '상 품X', '상 품H', '상 품C',
       '상품 B', '상품 O', '상품 T', '상품v', '상품p', '상품i', '상품 w', '상 품 s',
       '상 품 q', '상품s', '상품l', '상품t', '상품k', '상품g', '상품o', '상품 R', '상품 S',
       '상 품M', '상품j', '상품d', '상품 I', '상품 E', '상품 o', '상품c', '상품 v', '상품e',
       '상품x', '상 품I', ' 상품 W', ' 상품 X', '상품 M', '상 품P'], dtype=object)

In [42]:
# 매출이력 상품명 형식 통일 -> 공백 제거 및 대문자화

uriage['item_name'] = uriage['item_name'].map(lambda x: x.replace(' ',''))
uriage['item_name'] = uriage['item_name'].map(lambda x: x.upper())
print(uriage['item_name'].unique())
print(len(uriage['item_name'].unique()))

['상품A' '상품S' '상품Z' '상품V' '상품O' '상품U' '상품L' '상품C' '상품I' '상품R' '상품X' '상품G'
 '상품P' '상품Q' '상품Y' '상품N' '상품W' '상품E' '상품K' '상품B' '상품F' '상품D' '상품M' '상품H'
 '상품T' '상품J']
26


### 금액 결측치 수정

In [43]:
#test
import pandas as pd
df = pd.DataFrame([{"country":"한국","population":500},{"country":"미국","population":450},{"country":"싱가폴","population":705},
                   {"country":"호주","population":878},{"country":"베트남","population":660},{"country":"대만","population":808}])
df

,country,population
0,한국,500
1,미국,450
2,싱가폴,705
3,호주,878
4,베트남,660
5,대만,808


In [44]:
#test
flg = df['country'] == '한국'
flg

0     True
1    False
2    False
3    False
4    False
5    False
Name: country, dtype: bool

In [45]:
#교수님 버전
#특이사항 : 집계 기간에 상품 단가의 변동이 없다는 전제조건이 있음
flg_is_null = uriage['item_price'].isnull()
for idx, trg in enumerate(list(uriage.loc[flg_is_null,'item_name'].unique())):
    print(idx, trg)

for trg in list(uriage.loc[flg_is_null, 'item_name'].unique()):
    price = uriage.loc[(~flg_is_null)&(uriage['item_name']==trg),'item_price'].max()
    uriage['item_price'].loc[(flg_is_null) & (uriage['item_name']==trg)] = price
uriage.head()

0 상품S
1 상품A
2 상품P
3 상품N
4 상품W
5 상품R
6 상품I
7 상품L
8 상품F
9 상품O
10 상품B
11 상품C
12 상품V
13 상품Q
14 상품U
15 상품K
16 상품T
17 상품X
18 상품E
19 상품M
20 상품G
21 상품J
22 상품D
23 상품H
24 상품Y


,purchase_date,item_name,item_price,customer_name,month_year
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


In [46]:
for trg in list(uriage['item_name'].sort_values().unique()):
    print(trg + '의최고가:' + str(uriage.loc[uriage['item_name']==trg]['item_price'].max())
                          +'의 최저가:'+str(uriage.loc[uriage['item_name']==trg]['item_price'].min(skipna=False)))


상품A의최고가:100.0의 최저가:100.0
상품B의최고가:200.0의 최저가:200.0
상품C의최고가:300.0의 최저가:300.0
상품D의최고가:400.0의 최저가:400.0
상품E의최고가:500.0의 최저가:500.0
상품F의최고가:600.0의 최저가:600.0
상품G의최고가:700.0의 최저가:700.0
상품H의최고가:800.0의 최저가:800.0
상품I의최고가:900.0의 최저가:900.0
상품J의최고가:1000.0의 최저가:1000.0
상품K의최고가:1100.0의 최저가:1100.0
상품L의최고가:1200.0의 최저가:1200.0
상품M의최고가:1300.0의 최저가:1300.0
상품N의최고가:1400.0의 최저가:1400.0
상품O의최고가:1500.0의 최저가:1500.0
상품P의최고가:1600.0의 최저가:1600.0
상품Q의최고가:1700.0의 최저가:1700.0
상품R의최고가:1800.0의 최저가:1800.0
상품S의최고가:1900.0의 최저가:1900.0
상품T의최고가:2000.0의 최저가:2000.0
상품U의최고가:2100.0의 최저가:2100.0
상품V의최고가:2200.0의 최저가:2200.0
상품W의최고가:2300.0의 최저가:2300.0
상품X의최고가:2400.0의 최저가:2400.0
상품Y의최고가:2500.0의 최저가:2500.0
상품Z의최고가:2600.0의 최저가:2600.0


In [47]:
#강희버전
# uriage['item_price'] = uriage['item_price'].fillna(uriage.groupby('item_name')['item_price'].transform(max))
# uriage['item_price'].unique()

In [48]:
uriage['item_price'].isnull().sum()

0

### 고객이름 수정

In [49]:
kokyaku.head()

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


In [50]:
kokyaku['고객이름'] = kokyaku['고객이름'].map(lambda x :x.replace(' ',''))
kokyaku['고객이름'].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

### 등록일 datetime 변경(날짜오류 수정)

In [51]:
flg_is_serial = kokyaku['등록일'].astype('str').str.isdigit()
flg_is_serial.sum()

22

In [52]:
kokyaku.loc[flg_is_serial]

,고객이름,지역,등록일
1,김도윤,E시,42782
3,김하윤,F시,42872
4,김시온,E시,43127
21,김동우,A시,42920
27,김서준,C시,42901
47,김태민,H시,42741
49,김정원,H시,42929
53,김주영,E시,42833
76,김지성,A시,43188
80,김지후,D시,43110


In [53]:
fromSerial = pd.to_timedelta(kokyaku.loc[flg_is_serial,'등록일'].\
                             astype('float'), unit='D') + pd.to_datetime('1900/01/01')
fromSerial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]

In [54]:
fromString = pd.to_datetime(kokyaku.loc[~flg_is_serial,'등록일'])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [55]:
kokyaku['등록일'] = pd.concat([fromSerial,fromString])
kokyaku

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-18
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-19
4,김시온,E시,2018-01-29
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


In [56]:
kokyaku['등록연월'] = kokyaku['등록일'].dt.strftime('%Y%m')
rslt = kokyaku.groupby('등록연월')['고객이름'].count()
print(rslt)
print(len(kokyaku))

등록연월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64
200


In [57]:
# 등록일 컬럼에 숫자 데이터가 남아 있는지 확인
kokyaku['등록일'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 200 entries, 0 to 199
Series name: 등록일
Non-Null Count  Dtype         
--------------  -----         
200 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 1.7 KB


In [58]:
#pd.to_datetime(kokyaku['등록일'], errors='coerce')

In [59]:
kokyaku['등록일'].unique()[:5]

<DatetimeArray>
['2018-01-04 00:00:00', '2017-02-18 00:00:00', '2018-01-07 00:00:00',
 '2017-05-19 00:00:00', '2018-01-29 00:00:00']
Length: 5, dtype: datetime64[ns]

In [60]:
# 등록이리 컬럼에 숫자 데이터가 남아 있는지 확인
kokyaku['등록일'].astype('str').str.isdigit().sum()

0

### 8. 고객이름을 키로 두개의 데이터를 결합(조인)

In [65]:
# uriage, kokyaku
join_data = pd.merge(uriage, kokyaku, how = 'left', left_on='customer_name', right_on='고객이름').drop(['customer_name'], axis=1)

In [66]:
uriage

,purchase_date,item_name,item_price,customer_name,month_year
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904
...,...,...,...,...,...
2994,2019-02-15 02:56:00,상품Y,2500.0,김정민,201902
2995,2019-06-22 04:03:00,상품M,1300.0,김재원,201906
2996,2019-03-29 11:14:00,상품Q,1700.0,김지율,201903
2997,2019-07-14 12:56:00,상품H,800.0,김승주,201907


### 9.정제 데이터 덤프

In [68]:
dump_data = join_data[['purchase_date','month_year','item_name','item_price','고객이름','지역','등록일']]
dump_data

,purchase_date,month_year,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31
2996,2019-03-29 11:14:00,201903,상품Q,1700.0,김지율,B시,2017-03-15
2997,2019-07-14 12:56:00,201907,상품H,800.0,김승주,E시,2018-07-15


In [69]:
dump_data.to_csv(r'./Data-out/dump_data.csv', index = False)

### 10. 데이터를 집계

In [70]:
import_data = pd.read_csv(r'./Data-out/dump_data.csv')
import_data

,purchase_date,month_year,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31
2996,2019-03-29 11:14:00,201903,상품Q,1700.0,김지율,B시,2017-03-15
2997,2019-07-14 12:56:00,201907,상품H,800.0,김승주,E시,2018-07-15


In [71]:
# month_year를 세로축으로 해서 상품별로 집계
byItem = import_data.pivot_table(index='month_year',columns = 'item_name',aggfunc = 'size',fill_value = 0)
byItem

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
month_year,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [87]:
# 매출금액별 집계
byprice = import_data.pivot_table(index = 'month_year',columns = 'item_name', values='item_price',aggfunc = 'sum',fill_value =0)
byprice

item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,...,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
month_year,,,,,,,,,,,,,,,,,,,,,
201901,1800.0,2600.0,5700.0,6800.0,9000.0,9000.0,7700.0,12800.0,16200.0,17000.0,...,28900.0,37800.0,38000.0,34000.0,14700.0,48400.0,29900.0,33600.0,25000.0,0.0
201902,1900.0,2800.0,7800.0,8400.0,8000.0,8400.0,9800.0,13600.0,10800.0,14000.0,...,37400.0,39600.0,41800.0,46000.0,39900.0,48400.0,55200.0,38400.0,27500.0,2600.0
201903,1700.0,4200.0,6000.0,6800.0,4500.0,16200.0,9800.0,14400.0,10800.0,16000.0,...,39100.0,28800.0,38000.0,24000.0,48300.0,39600.0,36800.0,50400.0,40000.0,0.0
201904,1700.0,3800.0,7200.0,8000.0,9000.0,10200.0,9800.0,8800.0,16200.0,13000.0,...,34000.0,36000.0,30400.0,32000.0,23100.0,33000.0,32200.0,38400.0,50000.0,0.0
201905,2400.0,2800.0,4800.0,5600.0,9500.0,10800.0,16100.0,12000.0,14400.0,11000.0,...,22100.0,39600.0,34200.0,32000.0,33600.0,19800.0,48300.0,38400.0,50000.0,0.0
201906,2400.0,2400.0,3300.0,7600.0,6500.0,10800.0,10500.0,10400.0,17100.0,22000.0,...,25500.0,28800.0,39900.0,24000.0,37800.0,44000.0,39100.0,36000.0,32500.0,0.0
201907,2000.0,4000.0,5100.0,6800.0,6000.0,10200.0,13300.0,15200.0,17100.0,23000.0,...,25500.0,34200.0,43700.0,42000.0,27300.0,61600.0,36800.0,43200.0,30000.0,0.0


In [88]:
# 고객 집계
byCustomer = import_data.pivot_table(index = 'month_year',columns = '고객이름',aggfunc = 'size',fill_value =0)
byCustomer

고객이름,김가온,김강민,김강현,김건우,김건희,김경민,김규민,김규현,김다온,김대현,...,김현수,김현승,김현우,김현준,김현진,김호준,정도형,정영훈,정우석,정준기
month_year,,,,,,,,,,,,,,,,,,,,,
201901,1,2,1,2,5,3,1,1,1,5,...,3,2,4,2,1,1,4,4,4,3
201902,3,1,4,3,3,5,2,3,1,5,...,5,3,7,0,0,1,2,0,2,2
201903,4,0,3,1,1,2,2,1,1,5,...,3,2,1,2,1,2,1,2,3,4
201904,1,0,3,0,2,1,2,6,2,4,...,0,4,3,1,1,2,2,4,3,1
201905,0,0,2,2,1,4,6,3,3,1,...,2,2,1,4,3,0,0,0,1,2
201906,5,0,3,0,2,1,6,1,1,1,...,2,4,1,0,4,2,2,0,0,2
201907,3,1,3,2,2,1,4,1,4,3,...,2,1,0,4,0,2,6,1,2,3


In [90]:
# 지역 집계
byRegion = import_data.pivot_table(index = 'month_year',columns = '지역',aggfunc = 'size')
byRegion

지역,A시,B시,C시,D시,E시,F시,G시,H시
month_year,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54


In [127]:
# 집계 기간에 구매 이력이 없는 사용자 확인
# uriage, kokyaku : merge

away_data = pd.merge(uriage, kokyaku, how = 'right', left_on = 'customer_name',right_on='고객이름' )
away_data

,purchase_date,item_name,item_price,customer_name,month_year,고객이름,지역,등록일,등록연월
0,2019-02-24 01:07:00,상품C,300.0,김현성,201902,김현성,H시,2018-01-04,201801
1,2019-05-08 15:42:00,상품P,1600.0,김현성,201905,김현성,H시,2018-01-04,201801
2,2019-07-03 07:49:00,상품M,1300.0,김현성,201907,김현성,H시,2018-01-04,201801
3,2019-01-02 13:52:00,상품L,1200.0,김현성,201901,김현성,H시,2018-01-04,201801
4,2019-06-29 04:58:00,상품R,1800.0,김현성,201906,김현성,H시,2018-01-04,201801
...,...,...,...,...,...,...,...,...,...
2995,2019-02-17 06:40:00,상품S,1900.0,김시현,201902,김시현,H시,2019-04-19,201904
2996,2019-04-23 02:16:00,상품G,700.0,김시현,201904,김시현,H시,2019-04-19,201904
2997,2019-06-15 00:31:00,상품J,1000.0,김시현,201906,김시현,H시,2019-04-19,201904
2998,2019-07-17 23:00:00,상품K,1100.0,김시현,201907,김시현,H시,2019-04-19,201904


In [120]:
away_data[away_data['purchase_date'].isnull()][['고객이름','등록일']]

,고객이름,등록일
2999,김서우,2019-04-23
